In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import catboost 
import sklearn as sk 

from tqdm import tqdm
from time import time
import datetime
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt.plots import plot_convergence
from copy import deepcopy
import pprint
import shap
import os

import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
import warnings
warnings.filterwarnings("ignore")

# Classifiers
from catboost import CatBoostClassifier

# Model selection
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/op-dataset/OP_test.csv
/kaggle/input/op-dataset/OP.csv
/kaggle/input/dsba-fml-foundations-of-machine-learning/test.csv
/kaggle/input/dsba-fml-foundations-of-machine-learning/sample_submission.csv
/kaggle/input/dsba-fml-foundations-of-machine-learning/train.csv
/kaggle/input/dsba-fml-foundations-of-machine-learning/skeleton_code.py


In [2]:
train_data = pd.read_csv("../input/op-dataset/OP.csv")
test_data = pd.read_csv("../input/op-dataset/OP_test.csv")

In [3]:
#Removing the Unnamed: 0 column (This column is always)
train_data = train_data.drop('Unnamed: 0', axis=1)
test_data = test_data.drop('Unnamed: 0', axis=1)

train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

train_data = train_data.replace([np.inf, -np.inf], 0)
test_data = test_data.replace([np.inf, -np.inf], 0)

# train_data_copy = train_data

In [4]:
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

In [ ]:
# new_train = train_data_copy[~train_data_copy.duplicated(X_train.columns)]

In [ ]:
# X_train = new_train.drop('label', axis=1)
# y_train = new_train['label']

In [5]:
#Calculating Mutual Information gain for the columns of the train set
from sklearn.feature_selection import mutual_info_classif
mif = mutual_info_classif(X_train, y_train)

In [6]:
#Find the columns where the MF is more than 0.6
col_indexes = np.where(mif > 0.6)

In [ ]:
#The columns that will be used after capturing information gain
X_train = X_train.to_numpy()[:, col_indexes]
test_data = test_data.to_numpy()[:, col_indexes]

In [ ]:
#Reshaping the data 
X_train = X_train.reshape(48145, 31)
test_data = test_data.reshape(34365,31)

In [ ]:
test_data.shape

In [ ]:
#Data scaling 

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.decomposition import PCA

mms = StandardScaler()
X_train = mms.fit_transform(X_train)
test_data = mms.transform(test_data)

# pca = PCA(n_components=75)
# X_train = pca.fit_transform(X_train)
# test_data = pca.transform(test_data)

# lb = LabelBinarizer()
# y_train = lb.fit_transform(y_train)

In [ ]:
#LGBM model which was used as a baseline against Catboost 
import lightgbm as lgb


def f1_eval(preds, dtrain):
    labels = dtrain.get_label()
    preds = preds.reshape(len(np.unique(labels)), -1)
    preds = preds.T.argmax(axis = 1)
    f_score = f1_score(preds, labels, average="micro")
    return 'f1_score', f_score, True



dataset = lgb.Dataset(X_train, label=y_train)
test_dataset = lgb.Dataset(test_data)
params = {'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'n_jobs': -1,
          'num_leaves': 1000,
          'learning_rate': 0.026623466966581126,
          'max_depth': 500,
          'num_iterations': 1000,
          'lambda_l1': 2.959759088169741,
#           'lambda_l2': 1.331172832164913,
          'bagging_fraction': 0.9655406551472153,
          'bagging_freq': 5,
          'num_class': 8,
          'colsample_bytree': 0.6867118652742716,
         'learning_rate': 0.01}




#parameters dict for LightGBM
lgb_params =  {
    'boosting': 'dart', 
    'colsample_bytree': 1, 
    'learning_rate': 0.1, 
    'max_depth': 20, 
    'min_child_samples': 20, 
    'n_estimators': 1000, 
    'num_leaves': 500,  
    'objective': 'multiclass',
    'num_class':8,
    'reg_alpha': 0.6, 
    'reg_lambda': 0.3, 
    'subsample': 0.8,
    'verbose':1
    }


params_2 = {
    'application': 'multiclass', # for binary classification
    'num_class' : 8, # used for multi-classes
    'boosting_type': 'goss', # traditional gradient boosting decision tree
    'num_iterations': 600, 
    'learning_rate': 0.1,
    'num_leaves': 300,
    'device': 'gpu', # you can use GPU to achieve faster learning
    'max_depth': 0, # <0 means no limit
    # Small number of bins may reduce training accuracy but can deal with over-fitting
    'lambda_l1': 0.5, # L1 regularization
    'lambda_l2': 0.3, # L2 regularization
    'subsample_for_bin': 300, # number of samples for constructing bins
    'subsample': 1, # subsample ratio of the training instance
    'colsample_bytree': 0.8, # subsample ratio of columns when constructing the tree
    'min_split_gain': 0.5, # minimum loss reduction required to make further partition on a leaf node of the tree
    'min_child_weight': 1, # minimum sum of instance weight (hessian) needed in a leaf
    'min_child_samples': 5# minimum number of data needed in a leaf
}

## Bayesian Optimization for LGBM and Catboost 

In [ ]:
#Doing Bayesian Search for optimal parameters 

from bayes_opt import BayesianOptimization
from sklearn.model_selection import * 
from sklearn.metrics import *

#Define the Bayesian parameter search function
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=6,n_estimators=10000, output_process=False):
    # prepare data for parameter search 
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters to be searched for 
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, 
                 min_data_in_leaf,min_sum_hessian_in_leaf,subsample, lambda_l1, lambda_l2,
                min_split_gain, force_col_wise, force_row_wise, feature_fraction_bynode):
        params = {'application':'multiclass', 'metric':'multi_error','num_classes':8}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)
        params['lambda_l1'] = int(lambda_l1)
        params['lambda_l2'] = int(lambda_l2)
        params['min_split_gain'] = int(min_split_gain)
        params['force_col_wise'] = force_col_wise
        params['force_row_wise'] = force_row_wise
        params['feature_fraction_bynode'] = feature_fraction_bynode
        
        
        #Compute the CV scores and then give the max value for the multi_logloss_mean 
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['multi_logloss'])
        return max(cv_result['multi_logloss-mean'])
     
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.01, 1.0),
                                            'num_leaves': (100, 500),
                                            'feature_fraction': (0.1, 1.0),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 30),
                                            'max_bin':(20,200),
                                            'min_data_in_leaf': (20, 100),
                                            'min_sum_hessian_in_leaf':(0,100),
                                           'subsample': (0.01, 1.0),
                                           'lambda_l1': (0.01, 1.0),
                                           'lambda_l2': (0.01, 1.0),
                                            'min_split_gain':(0.01, 2.0),
                                            'force_col_wise':[True, False],
                                            'force_row_wise':[True,False],
                                            'feature_fraction_bynode':(0.05, 0.9)
                                           })

    
    #n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
    #init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    model_score=[]
    for model in range(len(lgbBO.res)):
        model_score.append(lgbBO.res[model]['target'])
    
    # return best parameters
    return lgbBO.res[pd.Series(model_score).idxmax()]['target'],lgbBO.res[pd.Series(model_score).idxmax()]['params']

opt_params = bayes_parameter_opt_lgb(X_train, y_train, init_round=20, opt_round=20, n_folds=5, random_seed=23,n_estimators=10000)

In [ ]:
#We find the best paramters 
bayesian_params = opt_params[1]

In [ ]:
#The dictionary of bayesian params 
bayesian_params

In [ ]:
#The parameters were float, so it needs to be changed to int type 
bayesian_params['num_leaves'] = int(round(bayesian_params['num_leaves']))
bayesian_params['max_bin'] = int(round(bayesian_params['max_bin']))
bayesian_params['max_depth'] = int(round(bayesian_params['max_depth']))
bayesian_params['min_data_in_leaf'] = int(round(bayesian_params['min_data_in_leaf']))

In [ ]:
#The extra parameters 
bayesian_params['objective'] = "multiclass"
bayesian_params['metric'] = "multi_error"
bayesian_params['num_classes'] = 8
bayesian_params['num_iterations'] = 1000

In [ ]:
#Train LGBM on these new parameters
dataset = lgb.Dataset(X_train, y_train)
mod = lgb.train(params_2, dataset, valid_sets=[dataset], feval=f1_eval,early_stopping_rounds = 50)

In [ ]:
pred_train = mod.predict(X_train)

In [ ]:
pred_train = pred_train.round().argmax(axis=1)

In [ ]:
f1_score(y_train, pred_train, average='micro')

In [ ]:
test_preds = mod.predict(test_data)

In [ ]:
test_preds = prediction.round().argmax(axis=1)

In [ ]:
np.unique(test_preds)

In [ ]:
test_preds

In [ ]:
pd.DataFrame(test_preds).to_csv("lgbm.csv")

In [ ]:
#Model Training using SKF approach

folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=31416)
oof = np.zeros(len(train_data))
predictions = np.zeros(len(test_data))
# feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(X_train[trn_idx], label=y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], label=y_train[val_idx])

    num_round = 15000
    clf = lgb.train(bayesian_params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=500, early_stopping_rounds = 250)
    predictions = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    oof[val_idx] = predictions.round().argmax(axis=1)
    
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["Feature"] = features
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = fold_ + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
#     predictions += clf.predict(test_data, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(f1_score(y_train, oof, average="micro")))

 ## CatBoost Model

In [ ]:
#Model Optimizer class for optimizing catboost through cross-validation
class ModelOptimizer:
    best_score = None
    opt = None
    
    def __init__(self, model, Xtrain, ytrain, categorical_columns_indices=None, n_fold=5, seed=1000, early_stopping_rounds=30, is_stratified=True, is_shuffle=True):
        self.model = model
        self.Xtrain = Xtrain
        self.ytrain = ytrain
        self.categorical_columns_indices = categorical_columns_indices
        self.n_fold = n_fold
        self.seed = seed
        self.early_stopping_rounds = early_stopping_rounds
        self.is_stratified = is_stratified
        self.is_shuffle = is_shuffle
        
        
    def update_model(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self.model, k, v)
            
    def evaluate_model(self):
        pass
    
    def optimize(self, param_space, max_evals=10):
#         start_time = time.time()
        
        @use_named_args(param_space)
        def _minimize(**params):
            self.model.set_params(**params)
            return self.evaluate_model()
        
        opt = gp_minimize(_minimize, param_space, n_calls=max_evals, random_state=2405, n_jobs=-1)
        best_values = opt.x
        optimal_values = dict(zip([param.name for param in param_space], best_values))
        best_score = opt.fun
        self.best_score = best_score
        self.opt = opt
        
        print('optimal_parameters: {}\noptimal score: {}\noptimization time: {}'.format(optimal_values, best_score))
        print('updating model with optimal values')
        self.update_model(**optimal_values)
        plot_convergence(opt)
        return optimal_values
    
class CatboostOptimizer(ModelOptimizer):
    #The catboostoptimizer class will inherit from ModelOptimizer and run 5-fold CV with the best parameters that have bee 
    #searched, and then return the best scores. 
    def evaluate_model(self):
        validation_scores = catboost.cv(
        catboost.Pool(self.Xtrain, 
                      self.ytrain, 
                      cat_features=self.categorical_columns_indices),
        params=self.model.get_params(), 
        nfold=self.n_fold,
        stratified=self.is_stratified,
        seed=self.seed,
        early_stopping_rounds=self.early_stopping_rounds,
        shuffle=self.is_shuffle,
        verbose_eval=False,
        plot=False)
        self.scores = validation_scores
        test_scores = validation_scores.iloc[:, 2]
        best_metric = test_scores.max()
        return 1 - best_metric

In [ ]:
#Since catboost runs best on categorical columns that are pre-specified, these are the categorical columns
cat_cols = ["month","weekday","designation","No_sender_reported","update_org",
           "personal_org","promotions_org","forums_org","purchases_org","travel_org","spam_org","social_org","update_tld",
            "personal_tld","promotions_tld","forums_tld","purchases_tld","spam_tld","social_tld","forum_sender","social_sender",
            "promo_sender","update_sender","update_sender_only","promo_sender_only","forum_sender_only","social_sender_only",
            "promo_and_update_sender"]

In [ ]:
#Initial Catboost Cross validation run using randomly chosen parameters 

params_cb = {
    "iterations": 600,
    "learning_rate":0.01,
    "depth":8,
    "l2_leaf_reg":0.5,
    "loss_function":"MultiClass",
    "custom_metric":"F1",
    "min_child_samples":100,
    "bootstrap_type":"Bayesian",
    "auto_class_weights":"Balanced",
    "classes_count":8
}



grid_params = {'min_data_in_leaf': 600,
  'depth': 4,
#   'bagging_temperature': 42.68421052631579,
  'rsm': 0.43157894736842106,
#   'random_strength': 58.315789473684205,
  'learning_rate': 0.16631578947368422,
  'l2_leaf_reg': 0.8,
    'bootstrap_type':"Bayesian",
    'auto_class_weights':"Balanced",
    'classes_count':8,
    'loss_function':"MultiClass",
    'custom_metric':"F1"
        }


cv_dataset = catboost.Pool(data=X_train, label=y_train)

cv = catboost.cv(pool=cv_dataset, params=params_cb, iterations=500, 
                fold_count=5, nfold=5, shuffle=True, stratified=True,
                verbose_eval=True, plot=True)

In [ ]:
#Feature Importance calculated using Shapley Values (Was dropped later due to high computation cost and memory overflow)
catboost.get_feature_importance(cv_dataset, type='ShapValues')

# CatBoost Hyperparameter Tuning using Bayesian search

In [ ]:
from catboost import Pool, cv, CatBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import * 
from sklearn.metrics import *


#Catboost optimization using Bayesian parameter search
def CB_opt(n_estimators, depth, learning_rate, max_bin, max_leaves, min_child_samples, l2_leaf_reg, model_size_reg, bagging_temperature,
          random_strength): 
    scores = []
    #Run stratifiedKfold to get the train and validation scores 
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1944)
    for train_index, test_index in skf.split(X_train, y_train):

        trainx, valx = X_train.iloc[train_index], X_train.iloc[test_index]
        trainy, valy = y_train.iloc[train_index], y_train.iloc[test_index]
        
        #Instantiate the class 
        reg = CatBoostClassifier(verbose = 0,
                                n_estimators = int(n_estimators),
                                learning_rate = learning_rate,
#                                 subsample = subsample, 
                                l2_leaf_reg = l2_leaf_reg,
                                max_depth = int(depth),
                                min_child_samples = int(min_child_samples),
                                bagging_temperature = bagging_temperature,
                                random_strength = random_strength,
                                max_leaves = int(max_leaves),
                                random_state = 100,
                                grow_policy = "Lossguide",
                                max_bin = int(max_bin),  
                                use_best_model = True, 
                                model_size_reg = model_size_reg,
                                auto_class_weights = "Balanced",
                                classes_count = 8

                                )
        #Fit the model and let the training begin, evaluate the prediction using F1-micro score. 
        reg.fit(trainx, trainy, eval_set = (valx, valy))
        scores.append(f1_score(valy, reg.predict(valx), average="micro"))
    return np.mean(scores)

# pbounds = {"n_estimators": (150,1000),
#            "depth": (2,10),
#            "learning_rate": (.01, 0.2),
#            "subsample":(0.4, 1.),
#            "num_leaves": (16,40),
#            "max_bin":(150,300),
#            "l2_leaf_reg":(0,10),
#            "model_size_reg": (0,10)
# }
#The search space defined for the model.
search_spaces = {'n_estimators': (200, 800),
                 'depth': (6, 8),
#                  'subsample':(0.1, 1.0),
                 'learning_rate': (0.01, 1.0),
                 'random_strength': (1e-9, 10),
                 'bagging_temperature': (0.0, 1.0),
                 'l2_leaf_reg': (2, 30),
                 'min_child_samples': (100, 400),
                 'max_leaves':(10,40),
                 'max_bin':(50, 200),
                 'model_size_reg':(0, 10)
#                  'classes_count':8
                }
optimizer = BayesianOptimization(f = CB_opt, pbounds = search_spaces, verbose = 2,random_state = 888)

print(optimizer.maximize(init_points = 5, n_iter = 10))


In [ ]:
#Finding optimal paramters
from sklearn.utils.class_weight import compute_class_weight

#Calculate the class weights 
uniques = np.unique(y_train)
weights = compute_class_weight('balanced',uniques, y_train)


#Train the model 
model = catboost.CatBoostClassifier(iterations=1000,class_weights=weights,use_best_model=True
                                    , custom_metric="TotalF1", loss_function="MultiClass",
                                   od_type="Iter", grow_policy="Lossguide")

default_cb_optimizer = CatboostOptimizer(model, X_train, y_train)


params_space = [Real(0.01, 1.0, name='learning_rate'), 
                Integer(2, 12, name='max_depth'), 
                Real(0.4, 1.0, name='colsample_bylevel'), 
                Real(0.0, 100, name='bagging_temperature'), 
                Real(0.0, 100, name='random_strength'), 
                Real(1.0, 100, name='l2_leaf_reg'),
                Integer(100, 500, name='min_child_samples'),
                Integer(20, 50, name='max_leaves')]
# optimal_values = default_cb_optimizer.optimize(params_space, max_evals=40)

In [ ]:
#Optimize the model (Took too much time, so skipped it.)
optimal_values = default_cb_optimizer.optimize(params_space, max_evals=40)

In [ ]:
model = catboost.CatBoost()


#Ran basic randomized search for Catboost to get the parameters 
params_space = {
    "learning_rate": np.linspace(0.01, 1.0, 20),
    "max_depth":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "colsample_bylevel": np.linspace(0.4, 1.0, 20),
    "random_strength":np.linspace(1.0, 100, 20),
    "l2_leaf_reg":np.linspace(0.1, 1.0, 10),
    "min_child_samples":[100, 200, 300, 400, 500, 600]
#     "max_leaves":[20, 25, 30, 35, 40, 45, 50]
}

#Basic randomized search 
model.randomized_search(params_space,
                  X_train,
                  y_train,
                  cv=5,
                  n_iter=10,
                  partition_random_seed=0,
                  calc_cv_statistics=True, 
                  search_by_train_test_split=True,
                  refit=True, 
                  shuffle=True, 
                  stratified=True, 
                  train_size=0.8,
                    verbose=True)

In [ ]:
#Compute the class weights for the training
from sklearn.utils.class_weight import compute_class_weight

uniques = np.unique(y_train.to_numpy())
weights = compute_class_weight(None,uniques, y_train)

In [ ]:
import catboost

params_cb = {
    "iterations": 600,
    "learning_rate":0.01,
    "depth":8,
    "l2_leaf_reg":0.5,
    "loss_function":"MultiClass",
    "eval_metric":"TotalF1",
    "min_child_samples":500,
    "bootstrap_type":"Bayesian",
    "auto_class_weights":"Balanced",
    "classes_count":8
}


# 'params': {'min_data_in_leaf': 600,
#   'depth': 4,
#   'rsm': 0.6526315789473685,
#   'random_strength': 58.315789473684205,
#   'learning_rate': 0.16631578947368422,
#   'l2_leaf_reg': 0.8},

#The final catboost model.
model = catboost.CatBoostClassifier(iterations=5000,custom_metric="F1", class_weights=weights, 
                                    loss_function="MultiClass"
                                    , learning_rate=0.01, depth=9, task_type='GPU', l2_leaf_reg=0.8,
                                   classes_count=8, min_child_samples=600, bootstrap_type="Bayesian", od_type="Iter")

model.fit(X_train, y_train, eval_set=(X_train, y_train), plot=True)

In [ ]:
len(y_train)

In [ ]:
#The final predictions after Catboost was done training 
preds_2 = model.predict(test_data)

In [ ]:
pd.Series(preds_2.reshape(-1)).value_counts()

In [ ]:
np.unique(preds_2)

In [ ]:
# model = CatBoostClassifier(iterations=600, learning_rate=0.01, depth=8, 
#                           l2_leaf_reg=0.5,loss_function="MultiClass",
#                            bagging_temperature=0.6,
#                           eval_metric="TotalF1",classes_count=8, verbose=2,
#                             class_weights=weights)

# default_cb_optimizer = CatboostOptimizer(model, X_train, y_train)
# default_cb_optimizer.evaluate_model()

In [ ]:
# def report_perf(optimizer, X, y, title, callbacks=None):
#     """
#     A wrapper for measuring time and performances of different optmizers
    
#     optimizer = a sklearn or a skopt optimizer
#     X = the training set 
#     y = our target
#     title = a string label for the experiment
#     """
#     start = time()
#     if callbacks:
#         optimizer.fit(X, y, callback=callbacks)
#     else:
#         optimizer.fit(X, y)
#     d=pd.DataFrame(optimizer.cv_results_)
#     best_score = optimizer.best_score_
#     best_score_std = d.iloc[optimizer.best_index_].std_test_score
#     best_params = optimizer.best_params_
#     print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
#            +u"\u00B1"+" %.3f") % (time() - start, 
#                                   len(optimizer.cv_results_['params']),
#                                   best_score,
#                                   best_score_std))    
#     print('Best parameters:')
#     pprint.pprint(best_params)
#     print()
#     return best_params

In [ ]:
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# clf = CatBoostClassifier(thread_count=2,
#                          loss_function='MultiClass',
#                          od_type = 'Iter',
#                          eval_metric = "TotalF1",
#                          verbose= True)

# # Defining your search space
# search_spaces = {'iterations': Integer(10, 1000),
#                  'depth': Integer(1, 8),
#                  'learning_rate': Real(0.01, 1.0, 'log-uniform'),
#                  'random_strength': Real(1e-9, 10, 'log-uniform'),
#                  'bagging_temperature': Real(0.0, 1.0),
#                  'border_count': Integer(1, 255),
#                  'l2_leaf_reg': Integer(2, 30),
#                  'scale_pos_weight':Real(0.01, 1.0, 'uniform')}





In [ ]:
# f1 = make_scorer(f1_score, average="micro")

# opt = BayesSearchCV(clf,
#                     search_spaces,
#                     scoring=f1,
#                     cv=skf,
#                     n_iter=200,
#                     n_jobs=-1,  # use just 1 job with CatBoost in order to avoid segmentation fault
#                     return_train_score=True,
#                     refit=True,
#                     optimizer_kwargs={'base_estimator': 'GP'},
#                     random_state=42)

# best_params = report_perf(opt, X_train, y_train,'CatBoost')